In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install pydantic


[notice] A new release of pip is available: 23.2 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import PyPDF2 
from langchain.text_splitter import CharacterTextSplitter
import warnings
warnings.filterwarnings('ignore')
import langchain

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS


# reading text from pdf and converting it into chunks

In [3]:
pdf_path = "/Users/damodargupta/Desktop/Intellify-project/docs/Big_Mac_Index.pdf"

def extract_text_from_pdf(pdf_path):
    text = ""
    pdf_file = open(pdf_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    for page in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page].extract_text()

    pdf_file.close()
    return text

text = extract_text_from_pdf(pdf_path)

In [4]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

# getting vectorstore by using FAISS as a database

In [5]:
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [6]:
extract_text_from_pdf("/Users/damodargupta/Desktop/Intellify-project/docs/Big_Mac_Index.pdf")

'Big\nMac\nIndex\nThe\nBig\nMac\nIndex\nis\na\nprice\nindex\npublished\nsince\n1986\nby\nThe\nEconomist\nas\nan\ninformal\nway\nof\nmeasuring\nthe\npurchasing\npower\nparity\n(PPP)\nbetween\ntwo\ncurrencies\nand\nproviding\na\ntest\nof\nthe\nextent\nto\nwhich\nmarket\nexchange\nrates\nresult\nin\ngoods\ncosting\nthe\nsame\nin\ndifferent\ncountries.\nIt\n"seeks\nto\nmake\nexchange-rate\ntheory\na\nbit\nmore\ndigestible."\nThe\nindex\ncompares\nthe\nrelative\nprice\nworldwide\nto\npurchase\nthe\nBig\nMac,\na\nhamburger\nsold\nat\nMcDonald\'s\nrestaurants.\nOverview\nThe\nBig\nMac\nindex\nwas\nintroduced\nin\nThe\nEconomist\nin\nSeptember\n1986\nby\nPam\nWoodall\nas\na\nsemi-humorous\nillustration\nof\nPPP\nand\nhas\nbeen\npublished\nby\nthat\npaper\nannually\nsince\nthen.\nAlthough\nthe\nBig\nMac\nIndex\nwas\nnot\nintended\nto\nbe\na\nlegitimate\ntool\nfor\nexchange\nrate\nevaluation,\nit\nis\nnow\nglobally\nrecognised\nand\nfeatured\nin\nmany\nacademic\ntextbooks\nand\nreports.\nThe\nin

In [7]:
get_text_chunks(text)

['Big\nMac\nIndex\nThe\nBig\nMac\nIndex\nis\na\nprice\nindex\npublished\nsince\n1986\nby\nThe\nEconomist\nas\nan\ninformal\nway\nof\nmeasuring\nthe\npurchasing\npower\nparity\n(PPP)\nbetween\ntwo\ncurrencies\nand\nproviding\na\ntest\nof\nthe\nextent\nto\nwhich\nmarket\nexchange\nrates\nresult\nin\ngoods\ncosting\nthe\nsame\nin\ndifferent\ncountries.\nIt\n"seeks\nto\nmake\nexchange-rate\ntheory\na\nbit\nmore\ndigestible."\nThe\nindex\ncompares\nthe\nrelative\nprice\nworldwide\nto\npurchase\nthe\nBig\nMac,\na\nhamburger\nsold\nat\nMcDonald\'s\nrestaurants.\nOverview\nThe\nBig\nMac\nindex\nwas\nintroduced\nin\nThe\nEconomist\nin\nSeptember\n1986\nby\nPam\nWoodall\nas\na\nsemi-humorous\nillustration\nof\nPPP\nand\nhas\nbeen\npublished\nby\nthat\npaper\nannually\nsince\nthen.\nAlthough\nthe\nBig\nMac\nIndex\nwas\nnot\nintended\nto\nbe\na\nlegitimate\ntool\nfor\nexchange\nrate\nevaluation,\nit\nis\nnow\nglobally\nrecognised\nand\nfeatured\nin\nmany\nacademic\ntextbooks\nand\nreports.\nThe\ni

In [8]:
text = text.replace("\n" , " ")

# cleaning the text

In [9]:
from nltk import sent_tokenize

sentences = sent_tokenize(text=text)
print(sentences)

['Big Mac Index The Big Mac Index is a price index published since 1986 by The Economist as an informal way of measuring the purchasing power parity (PPP) between two currencies and providing a test of the extent to which market exchange rates result in goods costing the same in different countries.', 'It "seeks to make exchange-rate theory a bit more digestible."', "The index compares the relative price worldwide to purchase the Big Mac, a hamburger sold at McDonald's restaurants.", 'Overview The Big Mac index was introduced in The Economist in September 1986 by Pam Woodall as a semi-humorous illustration of PPP and has been published by that paper annually since then.', 'Although the Big Mac Index was not intended to be a legitimate tool for exchange rate evaluation, it is now globally recognised and featured in many academic textbooks and reports.', 'The index also gave rise to the word burgernomics.', 'The theory underpinning the Big Mac index stems from the concept of PPP, which s

# writing appropriate prompt template for framing questions

In [10]:
from langchain.prompts import PromptTemplate

prompt_template_name  = PromptTemplate(
    input_variables= ['category' , 'sentence' , 'phrase'],
    template = "Generate a {category} {phrase} for {sentence}"
)

for sentence in sentences:
    prompt_template_name.format(category = 'one-word', phrase = 'question' , sentence = sentence)

In [11]:
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [12]:
import os
HUGGINGFACEHUB_API_TOKEN="YOUR_API_KEY" #insert your hugging face api key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [13]:
documents = ["/Users/damodargupta/Desktop/Intellify-project/docs/Big_Mac_Index.pdf"]

# generating questions

### 1. Multiple choice questions  (notice that answer is generated along with the question)

In [14]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5,"max_new_tokens":100, "max_length":10})
chain = LLMChain(llm = llm , prompt=prompt_template_name)
for i in range(2):
    print(chain.run({"category" : "multiple choice" , 'phrase' : 'question' , "sentence" : sentences[i]}))

What is the index used to measure?Options:A Exchange rates.B Purchasing power parity.C PPP.D Currency strength.Answer:D
What is the title of the book?Options:A Exchange Rate Theory.B Exchange Rates.C Exchange Rates and International Trade.D Exchange Rates and International Trade.Answer:B


### 2. one word answer questions

In [15]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5,"max_new_tokens":100, "max_length":10})
chain = LLMChain(llm = llm , prompt=prompt_template_name)
for i in range(2):
    print(chain.run({"category" : "one word answer" , 'phrase' : 'question' , "sentence" : sentences[i]}))

what is the big mac index?
What is the book about?


### 3. true false questions

In [20]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5,"max_new_tokens":100, "max_length":10})
chain = LLMChain(llm = llm , prompt=prompt_template_name)
for i in range(2):
    print(chain.run({"category" : " " , "phrase" : "statement that is either true or false" , "sentence" : sentences[i]}))

The Big Mac Index is a price index published by The Economist.
It is a book about the monetary system.


# generating answers

### 1. answers for the above generated true/false question

In [21]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5,"max_new_tokens":100, "max_length":10})
chain = LLMChain(llm = llm , prompt=prompt_template_name)
for i in range(2):
    print(chain.run({"category" : " " , "phrase" : "statement that is either true or false along with the answer" , "sentence" : sentences[i]}))

True
This book is about exchange rates


### 2. answer for a long question "explain big mac index"

In [9]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings

text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
chunks = text_splitter.split_text(text)

vectorstore = get_vectorstore(chunks)

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5 ,"max_new_tokens":100,"max_length":1000})
qa_retrieval_chain = RetrievalQA.from_chain_type(llm=llm, chain_type = 'stuff' ,retriever=vectorstore.as_retriever())
qa_retrieval_chain.run("explain big mac index?")

load INSTRUCTOR_Transformer
max_seq_length  512


'The Big Mac Index is a price index published since 1986 by The Economist as an informal way of measuring the purchasing power parity (PPP) between two currencies and providing a test of the extent to which market exchange rates result in goods costing the same in different countries. It "seeks to make exchange-rate theory a bit more digestible." The index compares the relative price worldwide to purchase the Big Mac, a hamburger sold at McDonald\'s restaurants'

### 3. answer for a one word question "what is big mac index?"

In [10]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings


text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
chunks = text_splitter.split_text(text)

vectorstore = get_vectorstore(chunks)

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5,"max_new_tokens":100 ,"max_length":100000})
qa_retrieval_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())
qa_retrieval_chain.run("answer in one word to what is big mac index?")

load INSTRUCTOR_Transformer
max_seq_length  512


'price'